### The statement that calls multiple generators looks ugly.
In such cases, with multiple genertors lined up, yield can start to feel unintuitive and tedious

__Enter Toolz__
<br> Toolz by Matt Rocklin - http://toolz.readthedocs.io/en/latest/
<br> It makes streaming super easy - intuitive and concise !

For more examples and explanation from Elegant Scipy written by the brilliant ASPP faculty - https://github.com/elegant-scipy/notebooks/blob/master/notebooks/ch8.ipynb

In [1]:
import toolz as tz
from toolz import curry
import getspeeddata as gs

In [2]:
curried_get_frames = curry(gs.getframes)

### toolz.pipe
Pipe is simply syntactic sugar to make multiple function calls easy
<br> Passes a value through a sequence of functions - one by one

In [4]:
# This will do exactly as the previous call (without the added brackets)
# The function calls are cleaner and can be read from left to right - which is sooo much better
def pipeline(filename):
    pipe = tz.pipe(filename,
                gs.readxy,
                gs.consecutivexy1,
                gs.getdist,
                curried_get_frames(threshold=10, frames_per_sec=30)
               )
    return pipe

In [7]:
dirname = 'FishTrackingMultiple/'
for i in gs.CSVfileGrabber(dirname):
    pipeline(i)

Working on: Fish1
Of 16.267 seconds recording time, time spent with speed less than 10 is 12.833 seconds
Working on: Fish2
Of 600.367 seconds recording time, time spent with speed less than 10 is 554.267 seconds
Working on: Fish4
Of 599.567 seconds recording time, time spent with speed less than 10 is 511.933 seconds
Working on: Fish5
Of 598.733 seconds recording time, time spent with speed less than 10 is 379.467 seconds
Working on: Fish6
Of 16.267 seconds recording time, time spent with speed less than 10 is 15.133 seconds


## Whats this curry?
Curry = Haskell Brooks Curry 
<br> __"Currying"__ means partially evaluating a function and returning another function. 

In [8]:
# If you dont give all inputs to a python function, it becomes angry
sum()

TypeError: sum expected at least 1 arguments, got 0

### By currying, we are breaking down the evaluation of a function 
A curried function evaluates partially when you dont give it all the arguments, and fully when all arguments are available. 

### In python, functions can be passed around as any other object
built-in function `Map` : Returns an iterator that applies function to every item of iterable, yielding the results
<br>`map(function_to_apply, list_of_inputs)`

In [33]:
def getlen(text):
    return len(text.split())

In [16]:
strings = ['I am ok', 'I guess I will be ok']
map(getlen, strings)

In [11]:
for i in map(getlen, strings):
    print(i)

3
6


In [12]:
list(map(getlen, strings))

[3, 6]

If we were to think about how the `map` function is implemented

In [13]:
def a_map_function(myfunc, myseq):
    for x in myseq: 
        yield myfunc(x)

In [14]:
for i in a_map_function(getlen, strings):
    print(i)

3
6


### Task 3: Implement the in-built `filter` function
__filter__ - Returns those elements of iterable for which function returns true <br> `filter(function, sequence)`

__To Do__ - From the file 'FixationTaskToDo.txt', filter those lines that are tasks to be fixed (#FIXME)

In [20]:
def words(text):
    return text.startswith('#FIXME')
        
def myfilter(myfunc, myseq):
    for x in myseq:
        if myfunc(x):
            yield x

for i in myfilter(words, open('Textfiles/FixationTaskToDo.txt')):
    print(i)

#FIXME: Reset all counters when the subject name changes

#FIXME: The program crashes sometimes, possibility after overlaying the video on top of the gray screen

#FIXME: Look into why rewarded fixations appear to fall outside the allowed window.

#FIXME: randomize calibration spot during calibration

#FIXME: run a check to ensure that the monkey does not enter other locations other than the target in the response window

#FIXME: Make sure we align the trials to the screen flip

#FIXME: Randomly cycle through fixation locations which can be interrupted, should be resumed when done

#FIXME: Add feedback on the experimenter's plot on how many trials have been completed



### Now lets think about how curry would work 
This is a curried implementation of the same map function

In [37]:
import functools
def curried_map_function(myfunc, myseq=None):
    if myseq == None:
        print('Partial')
        return functools.partial(map, myfunc)
    else:
        print('Yield')
        return map(myfunc, myseq) # Use a single sequence for now

In [40]:
get_all_length = curried_map_function(getlen, ['Ok'])

Yield


## If we want to curry any other generic function
For example, `add` that takes two arguments

In [48]:
# #def chain_curry(my_function):
#     """ A chain curry function """
#     def f1(a):
#         def f2(b):
#             return my_function(a, b)
#         return f2
#     return f1


def chain_curry(my_function):
    print('Entering the function')
    def f1(a):
        print('Evaluating partially given input', a)
        def f2(b):
            print('Evaluating fully after getting input', b)
            return my_function(a, b)
        print('f2', f2)
        return f2
    print('f1', f1)
    return f1


def add(a, b):
    return a + b

In [49]:
f = chain_curry(my_function=add) #Define a curried function

Entering the function
f1 <function chain_curry.<locals>.f1 at 0x108eb9ae8>


In [50]:
g = f(1) #Just holds on to the value and produces no error

Evaluating partially given input 1
f2 <function chain_curry.<locals>.f1.<locals>.f2 at 0x108eb9400>


In [51]:
g(5)

Evaluating fully after getting input 5


6

In [45]:
f(1)(5)

6

Problem with a chain curry?

`@chain_curry` is just syntactic sugar for `chain_curry(func1)`
<br> The `@` operator at the start of the line runs a decorator

In [52]:
@curry
def add(x, y):
    print(f'Sum {x + y}')
    return x + y

In [55]:
add(5)(3)

Sum 8


8

In [ ]:
add(a = 3, b = 5) #Why doesnt this work?

### The "actual" curry implementation

In [ ]:
def genericcurry(my_function):
    #     print('Going in')
    def new_func(*args, **kwargs):
        try:
            return my_function(*args, **kwargs)
        except TypeError:
            #             print('Not all arguments given')
            return functools.partial(new_func, *args, **kwargs)
    return new_func

In [ ]:
@genericcurry
def add(a, b, c):
    print(f'Sum {a+b+c}')
    return a + b + c

In [ ]:
f = add(b = 1, c = 2)
print(f)

In [ ]:
f(2)